<a href="https://colab.research.google.com/github/niz11/Bachelor_Thesis_Colab_Files/blob/main/10_frame_seq_classifier_RepeatVector_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load encoding
X = np.load('drive/My Drive/facea_seqs_one_frame_to_the_right/X.npy')
y = np.load('drive/My Drive/facea_seqs_one_frame_to_the_right/Y_label.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


In [ ]:
import keras
from keras.layers import Input, Flatten, Dense,LSTM,TimeDistributed,RepeatVector
from keras.models import Model,Sequential
from keras.utils import plot_model


def create_model(encoding_length=64,sequence_length=10):
  visible = Input(shape=(sequence_length,encoding_length))
  encoder_outputs, state_h, state_c = LSTM(encoding_length, activation='relu', name='encoder', return_state=True)(visible)
  encoder_state = [state_h, state_c]
  clssifier  = Dense(7, activation='softmax')(encoder_outputs)

  model = Model(inputs=visible, outputs=[clssifier])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  plot_model(model, show_shapes=True, to_file='lstm_autoencoder.png')
  model.summary()
  return model

model = create_model(128, 10)
model.load_weights("drive/My Drive/trained_models/composite_repeat_one_frame_to_the_right-new/encoderOneFrameToRightRepeat.h5",by_name=True)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10, 128)]         0         
_________________________________________________________________
encoder (LSTM)               [(None, 128), (None, 128) 131584    
_________________________________________________________________
dense (Dense)                (None, 7)                 903       
Total params: 132,487
Trainable params: 132,487
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train, epochs=5, verbose=1, validation_split=0.05)

Epoch 1/5
383/383 [==============================] - 12s 30ms/step - loss: 0.7649 - accuracy: 0.8007 - val_loss: 0.6245 - val_accuracy: 0.8388
Epoch 2/5
383/383 [==============================] - 12s 31ms/step - loss: 0.4863 - accuracy: 0.8569 - val_loss: 0.4625 - val_accuracy: 0.8682
Epoch 3/5
383/383 [==============================] - 12s 30ms/step - loss: 0.3712 - accuracy: 0.8956 - val_loss: 0.3748 - val_accuracy: 0.9085
Epoch 4/5
383/383 [==============================] - 11s 30ms/step - loss: 0.3109 - accuracy: 0.9122 - val_loss: 0.3073 - val_accuracy: 0.9163
Epoch 5/5
383/383 [==============================] - 12s 31ms/step - loss: 0.2386 - accuracy: 0.9324 - val_loss: 0.2461 - val_accuracy: 0.9349


In [ ]:
def getFaceExpressionFromIndex(i):
  if (i == 0):
      return 'surprise'
  elif (i == 1):
      return 'smile'
  elif (i == 2):
      return 'sad'
  elif (i == 3):
      return 'anger'
  elif (i == 4):
      return 'fear'
  elif (i == 5):
      return 'disgust'
  elif (i == 6):
      return 'none'
  else:
      print(i)

In [ ]:
test_predictions = model.predict(X_test)
correct = 0
for i in range(len(test_predictions)):
  truth = np.argmax(y_test[i])
  prediction = np.argmax(test_predictions[i])
  if truth == prediction:
    correct += 1
print(f'number of samples: {len(test_predictions)}')
print(f'correct: {correct}')
print(f'Accuracy of predicitons: {correct / len(test_predictions)}')

number of samples: 5528
correct: 5057
Accuracy of predicitons: 0.9147973950795948


In [ ]:
# Below start classifing the encodings

In [ ]:
from keras.models import load_model
from keras.models import model_from_json
import json

# load json and create model
json_file = open('drive/My Drive/trained_models/composite_repeat_one_frame_to_the_right-new/encoderOneFrameToRightRepeat.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
encoder = model_from_json(loaded_model_json)
# load weights into new model
encoder.load_weights("drive/My Drive/trained_models/composite_repeat_one_frame_to_the_right-new/encoderOneFrameToRightRepeat.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
# Check here: better divinding to test/train set
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Load encoding
X = np.load('drive/My Drive/facea_seqs_one_frame_to_the_right/X.npy')
y = np.load('drive/My Drive/facea_seqs_one_frame_to_the_right/Y_label.npy')
predictions = encoder.predict(X)


X_train, X_test, y_train, y_test = train_test_split(predictions, y, test_size = 0.3, random_state = 0)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


def classifier():
  model = Sequential()
  model.add(Dense(64, input_dim=128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(7, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


def classifier():
  model = Sequential()
  model.add(Dense(64, input_dim=128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(7, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model = classifier()
model.fit(X_train, y_train, epochs=50, verbose=1, validation_split=0.1)

Epoch 1/50
363/363 [==============================] - 1s 4ms/step - loss: 1.6698 - accuracy: 0.3518 - val_loss: 1.4986 - val_accuracy: 0.4364
Epoch 2/50
363/363 [==============================] - 1s 4ms/step - loss: 1.3791 - accuracy: 0.5019 - val_loss: 1.3009 - val_accuracy: 0.5419
Epoch 3/50
363/363 [==============================] - 1s 4ms/step - loss: 1.1988 - accuracy: 0.5824 - val_loss: 1.1385 - val_accuracy: 0.6109
Epoch 4/50
363/363 [==============================] - 1s 4ms/step - loss: 1.0517 - accuracy: 0.6381 - val_loss: 1.0444 - val_accuracy: 0.6457
Epoch 5/50
363/363 [==============================] - 1s 3ms/step - loss: 0.9078 - accuracy: 0.6956 - val_loss: 0.8879 - val_accuracy: 0.7101
Epoch 6/50
363/363 [==============================] - 1s 3ms/step - loss: 0.7786 - accuracy: 0.7419 - val_loss: 0.8250 - val_accuracy: 0.7326
Epoch 7/50
363/363 [==============================] - 1s 4ms/step - loss: 0.6689 - accuracy: 0.7837 - val_loss: 0.6972 - val_accuracy: 0.7752
Epoch 

In [ ]:
def getFaceExpressionFromIndex(i):
  if (i == 0):
      return 'surprise'
  elif (i == 1):
      return 'smile'
  elif (i == 2):
      return 'sad'
  elif (i == 3):
      return 'anger'
  elif (i == 4):
      return 'fear'
  elif (i == 5):
      return 'disgust'
  elif (i == 6):
      return 'none'
  else:
      print(i)

In [ ]:
test_predictions = model.predict(X_test)
correct = 0
for i in range(len(test_predictions)):
  truth = np.argmax(y_test[i])
  prediction = np.argmax(test_predictions[i])
  if truth == prediction:
    correct += 1
  else:
    print(f'Wrong classification, truth: {getFaceExpressionFromIndex(truth)}')
    print(f'Wrong classification, prediction: {getFaceExpressionFromIndex(prediction)}')
    print("---------------------------------------------------------------------------")

print(f'Accuracy of predicitons: {correct / len(test_predictions)}')
print(f'Got correct: {correct}')
print(f'Got wrong: {len(test_predictions) - correct}')

Wrong classification, truth: fear
Wrong classification, prediction: sad
---------------------------------------------------------------------------
Wrong classification, truth: disgust
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: fear
Wrong classification, prediction: smile
---------------------------------------------------------------------------
Wrong classification, truth: anger
Wrong classification, prediction: sad
---------------------------------------------------------------------------
Wrong classification, truth: sad
Wrong classification, prediction: none
---------------------------------------------------------------------------
Wrong classification, truth: disgust
Wrong classification, prediction: anger
---------------------------------------------------------------------------
Wrong classification, truth: smile
Wrong classification, prediction: none
--------------------------

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(12897, 128)
(5528, 128)
(12897, 7)
(5528, 7)
